In [1]:
import pandas as pd
import numpy as np
import sklearn
import math as mt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

from operator import itemgetter
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds

In [2]:
def DCGp(i, p, rel):
    tot = 0
    for i in range(p):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [3]:
def IDCGp(i, rel):
    tot = 0
    for i in range(abs(rel)):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [4]:
def nDCGp(i, p, rel):
    return DCGp(i, p, rel)/IDCGp(i, rel)

In [5]:
DF = pd.read_csv("citeulike-a/raw-data.csv",encoding='ISO-8859-1')#'ISO-8859-1')

In [6]:
DF.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...


In [7]:
DF.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...


In [8]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 5 columns):
doc.id          16980 non-null int64
title           16980 non-null object
citeulike.id    16980 non-null float64
raw.title       16980 non-null object
raw.abstract    16980 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 663.4+ KB


In [9]:
citations = pd.read_csv("citeulike-a/citations.dat", header = None, delimiter='\n')

In [10]:
citations.columns = ["citations"]

In [11]:
citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 1 columns):
citations    16980 non-null object
dtypes: object(1)
memory usage: 132.7+ KB


In [12]:
citations.head()

,citations
0,3 2 485 3284
1,16 42 43 60 113 116 161 252 1543 1782 1947 414...
2,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
3,0
4,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...


In [13]:
citations2=citations[1:len(citations)-1].copy()

In [14]:
#citations2.index.rename('doc.id',inplace=True)

In [15]:
citations2.reset_index(inplace=True)

In [16]:
citations2.tail()

,index,citations
16973,16974,12 1631 10468 10587 11949 12665 13198 14527 14...
16974,16975,16 420 6357 6371 9466 12007 12096 12660 13307 ...
16975,16976,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,0


In [17]:
DF2=pd.concat([DF,citations2],axis=1)
DF2

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...
5,6,random graphs with arbitrary degree distributi...,48.0,Random graphs with arbitrary degree distributi...,Recent work on the structure of social network...,6.0,14 49 1533 3646 7689 8302 10608 11649 12757 12...
6,7,artificial gene networks for objective compari...,49.0,Artificial gene networks for objective compari...,Motivation: Large-scale gene expression profil...,7.0,1 169
7,8,the segment polarity network is a robust devel...,50.0,The segment polarity network is a robust devel...,"All insects possess homologous segments, but s...",8.0,0
8,9,the evolutionary origin of complex features,52.0,The evolutionary origin of complex features.,A long-standing challenge to evolutionary theo...,9.0,9 3738 3757 4263 4298 6043 7014 9311 10171 15763
9,10,early language acquisition cracking the speech...,60.0,Early language acquisition: cracking the speec...,"Infants learn language with remarkable speed, ...",10.0,10 2 15 477 1200 1217 3937 5058 5561 8649 8958


In [18]:
type(DF2['citations'][0])

str

In [19]:
N = 10

In [20]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...


In [21]:
DF2.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...,16976.0,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...,16977.0,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...,16978.0,0
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...,NaN,NaN
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...,NaN,NaN


In [22]:
DF2.dropna(inplace=True,axis=0)

In [23]:
DF2['citations']=DF2['citations'].apply(lambda x: x.split(','))

In [24]:
del DF
del citations
del citations2

In [25]:
IdFreq={}

In [26]:
def docIdFreq (c_list):
    for element in c_list:
        if element in IdFreq:
            IdFreq[element] = IdFreq[element]+1
        else:
            IdFreq[element] = 1

In [27]:
DF2['citations'].apply(lambda x: docIdFreq(x));

In [28]:
most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)

In [29]:
del most_popular[0]

In [30]:
most_popular[0:N]

[('1 2216', 8),
 ('1 13117', 7),
 ('1 11229', 5),
 ('1 5943', 4),
 ('1 8717', 4),
 ('1 243', 4),
 ('1 1878', 4),
 ('1 965', 4),
 ('1 3803', 4),
 ('1 861', 3)]

In [31]:
DF2.drop('index',axis=1,inplace=True)

In [32]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,[16 42 43 60 113 116 161 252 1543 1782 1947 41...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,[85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 4...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",[0]
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,[23 2 28 488 918 1200 1351 1861 1937 2289 3079...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,[37 2 15 22 23 28 254 469 477 481 483 484 485 ...


In [33]:
DF2[DF2['doc.id']==1]['raw.title'].values[0]

'The metabolic world of Escherichia coli is not small'

# Most popular article 

'The metabolic world of Escherichia coli is not small'

In [34]:
UR = pd.read_csv("citeulike-a/users.dat",delimiter='\n',names=['liked_articles'],header=None)

In [35]:
UR.head()

,liked_articles
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...
3,12 706 709 721 755 756 776 892 895 2588 2991 4...
4,12 761 2678 4034 4035 5439 5494 5758 7798 9046...


In [36]:
UR['liked_articles']=UR['liked_articles'].apply(lambda x: x.split(' '))

In [37]:
UR.head()

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."


In [38]:
UR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5551 entries, 0 to 5550
Data columns (total 1 columns):
liked_articles    5551 non-null object
dtypes: object(1)
memory usage: 43.4+ KB


In [39]:
from sklearn.model_selection import KFold

In [40]:
kf = KFold(n_splits=5)
recommends = []


for train_index, validate_index in kf.split(UR):
    
    trainDF = pd.DataFrame(UR.loc[train_index])
    validateDF = pd.DataFrame(UR.loc[validate_index])
    print()
    
    IdFreq={}
    trainDF['liked_articles'].apply(lambda x: docIdFreq(x));
    most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)
    del most_popular[0]
    most_popular = most_popular[0:N]
    mp_docId = [ element[0] for element in most_popular]
    recommends.append(DF2[DF2['doc.id'].isin(mp_docId)]['raw.title'].values)
    
    


In [41]:
recommends

[array(['Organization, development and function of complex brain networks',
        'Motifs in brain networks.',
        'Diffusion on Complex Networks : A way to probe their large scale topological structures',
        'Topological Generalizations of network motifs',
        "Collective dynamics of 'small-world' networks.",
        'Network motifs: simple building blocks of complex networks.',
        'Authoritative sources in a hyperlinked environment',
        'Finding and evaluating community structure in networks',
        'Subgraph centrality in complex networks.',
        'Finding structure with randomness: Stochastic algorithms for constructing approximate matrix decompositions'], dtype=object),
 array(['Organization, development and function of complex brain networks',
        'Motifs in brain networks.',
        'Diffusion on Complex Networks : A way to probe their large scale topological structures',
        'Topological Generalizations of network motifs',
        "Collectiv

# Random

In [42]:
d_id=np.random.randint(low=0, high=len(DF2)-1,size=N)

In [43]:
DF2[DF2['doc.id'].isin(d_id)]['raw.title'].values

array([ 'Centralized channel assignment and routing algorithms for multi-channel wireless mesh networks',
       'Empirical and structural models for insertions and deletions in the divergent evolution of proteins.',
       'Toward Distributed Service Discovery in Pervasive Computing Environments',
       'Learning Eigenfunctions Links Spectral Embedding and Kernel PCA',
       'An alternative to the hand searching gold standard: validating methodological search filters using relative recall',
       'A document clustering and ranking system for exploring MEDLINE citations.',
       'Bipartite network projection and personal recommendation',
       'The John D. and Catherine T. MacArthur Foundation Series on Digital Media and Learning',
       'Cavity Optomechanics: Back-Action at the Mesoscale',
       'Design and analysis of ChIP-seq experiments for DNA-binding proteins.'], dtype=object)

# Content-Based

In [44]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [45]:
tfidf_matrix = tfidf_vectorizer.fit_transform(DF2['raw.abstract'])
nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine').fit(tfidf_matrix)

In [46]:
def get_closest_neighbors_tfidf(title):
    row                = DF2[DF2['raw.title']==title].index[0]
    distances, indices = nbrs_tfidf.kneighbors(tfidf_matrix.getrow(row))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    return result

In [47]:
get_closest_neighbors_tfidf('What is the goal of sensory coding')['raw.title'].values[1:N+1]

array([ 'Emergence of simple-cell receptive field properties by learning a sparse code for natural images',
       'Relations between the statistics of natural images and the response properties of cortical cells',
       'Sparse coding with an overcomplete basis set: A strategy employed by V',
       'Sparse coding and decorrelation in primary visual cortex during natural vision',
       'Natural image statistics and efficient coding',
       "The `Independent Components' of Natural Scenes are Edge Filters",
       'Cortical Sensitivity to Visual Features in Natural Scenes',
       'Statistical learning of new visual feature combinations by infants.',
       'Sparse Coding of Sensory Inputs',
       'Adaptive filtering enhances information transmission in visual cortex.'], dtype=object)

# Collaborative FiItering

In [48]:
docId = DF2['doc.id']

In [49]:
len(docId)

16978

In [50]:
for i in range(len(docId)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729


7552
7553
7554
7555
7556
7557
7558
7559
7560
7561
7562
7563
7564
7565
7566
7567
7568
7569
7570
7571
7572
7573
7574
7575
7576
7577
7578
7579
7580
7581
7582
7583
7584
7585
7586
7587
7588
7589
7590
7591
7592
7593
7594
7595
7596
7597
7598
7599
7600
7601
7602
7603
7604
7605
7606
7607
7608
7609
7610
7611
7612
7613
7614
7615
7616
7617
7618
7619
7620
7621
7622
7623
7624
7625
7626
7627
7628
7629
7630
7631
7632
7633
7634
7635
7636
7637
7638
7639
7640
7641
7642
7643
7644
7645
7646
7647
7648
7649
7650
7651
7652
7653
7654
7655
7656
7657
7658
7659
7660
7661
7662
7663
7664
7665
7666
7667
7668
7669
7670
7671
7672
7673
7674
7675
7676
7677
7678
7679
7680
7681
7682
7683
7684
7685
7686
7687
7688
7689
7690
7691
7692
7693
7694
7695
7696
7697
7698
7699
7700
7701
7702
7703
7704
7705
7706
7707
7708
7709
7710
7711
7712
7713
7714
7715
7716
7717
7718
7719
7720
7721
7722
7723
7724
7725
7726
7727
7728
7729
7730
7731
7732
7733
7734
7735
7736
7737
7738
7739
7740
7741
7742
7743
7744
7745
7746
7747
7748
7749
7750
7751


11568
11569
11570
11571
11572
11573
11574
11575
11576
11577
11578
11579
11580
11581
11582
11583
11584
11585
11586
11587
11588
11589
11590
11591
11592
11593
11594
11595
11596
11597
11598
11599
11600
11601
11602
11603
11604
11605
11606
11607
11608
11609
11610
11611
11612
11613
11614
11615
11616
11617
11618
11619
11620
11621
11622
11623
11624
11625
11626
11627
11628
11629
11630
11631
11632
11633
11634
11635
11636
11637
11638
11639
11640
11641
11642
11643
11644
11645
11646
11647
11648
11649
11650
11651
11652
11653
11654
11655
11656
11657
11658
11659
11660
11661
11662
11663
11664
11665
11666
11667
11668
11669
11670
11671
11672
11673
11674
11675
11676
11677
11678
11679
11680
11681
11682
11683
11684
11685
11686
11687
11688
11689
11690
11691
11692
11693
11694
11695
11696
11697
11698
11699
11700
11701
11702
11703
11704
11705
11706
11707
11708
11709
11710
11711
11712
11713
11714
11715
11716
11717
11718
11719
11720
11721
11722
11723
11724
11725
11726
11727
11728
11729
11730
11731
11732
11733
1173

15356
15357
15358
15359
15360
15361
15362
15363
15364
15365
15366
15367
15368
15369
15370
15371
15372
15373
15374
15375
15376
15377
15378
15379
15380
15381
15382
15383
15384
15385
15386
15387
15388
15389
15390
15391
15392
15393
15394
15395
15396
15397
15398
15399
15400
15401
15402
15403
15404
15405
15406
15407
15408
15409
15410
15411
15412
15413
15414
15415
15416
15417
15418
15419
15420
15421
15422
15423
15424
15425
15426
15427
15428
15429
15430
15431
15432
15433
15434
15435
15436
15437
15438
15439
15440
15441
15442
15443
15444
15445
15446
15447
15448
15449
15450
15451
15452
15453
15454
15455
15456
15457
15458
15459
15460
15461
15462
15463
15464
15465
15466
15467
15468
15469
15470
15471
15472
15473
15474
15475
15476
15477
15478
15479
15480
15481
15482
15483
15484
15485
15486
15487
15488
15489
15490
15491
15492
15493
15494
15495
15496
15497
15498
15499
15500
15501
15502
15503
15504
15505
15506
15507
15508
15509
15510
15511
15512
15513
15514
15515
15516
15517
15518
15519
15520
15521
1552

In [51]:
del CountVectorizer
del Counter
del IdFreq
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

In [52]:
DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

In [53]:
'''
M_train = np.zeros(shape=(len(UR),len(docId)))
M_test

for i, row in UR.iterrows():
    end = int(len(row['liked_articles'])*0.2)
    for j in range(end):
        M[i,int(row['liked_articles'][j])-1]=1
'''

"\nM_train = np.zeros(shape=(len(UR),len(docId)))\nM_test\n\nfor i, row in UR.iterrows():\n    end = int(len(row['liked_articles'])*0.2)\n    for j in range(end):\n        M[i,int(row['liked_articles'][j])-1]=1\n"

In [54]:
BARRIER();

NameError: name 'BARRIER' is not defined

In [55]:
UR = UR.iloc[0:3999]

In [56]:
UR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 1 columns):
liked_articles    3999 non-null object
dtypes: object(1)
memory usage: 31.3+ KB


### K-fold Cross Validation

In [57]:
from random import shuffle

In [58]:
def shuffleDF(UR):
    for i, row in UR.iterrows():
        x=row['liked_articles']
        shuffle(x)
        UR.loc[i,'liked_articles'] = x

In [59]:
UR.head()

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."


In [60]:
shuffleDF(UR)

In [61]:
UR.head()

,liked_articles
0,"[8903, 15894, 10204, 2846, 6968, 15300, 12684,..."
1,"[493, 13616, 8669, 14476, 2819, 15047, 1519, 1..."
2,"[7872, 4671, 1329, 4792, 800, 1984, 8800, 6385..."
3,"[895, 756, 776, 709, 2991, 2588, 721, 706, 755..."
4,"[5758, 4035, 5494, 4034, 12, 11877, 7798, 1190..."


In [62]:
def getTest(row,fold):
    
    UR_test=[]
    fold=fold-1
    pct=0.2
    readByUser = row['liked_articles'].copy()
    end = len(readByUser)
    end = int(end - (pct*fold))
    #print('readByUser', readByUser)
    pctValue=int(end*(pct))
    #print('pctValue: ',pctValue)
    readByUser = readByUser[end-pctValue:end+1]
    #print('readByUser Test', readByUser)
    #UR_train = np.delete(readByUser, np.s_[end-pct:end+1], axis=1)
    for element in readByUser:
        UR_test.append(int(element))
    return UR_test

In [63]:
def getTrain(UR, fold):
    
    M = np.zeros(shape=(len(UR),len(docId)+1))
    fold=fold-1
    pct=0.2

    for i, row in UR.iterrows():
        readByUser = row['liked_articles'].copy()
        end = len(readByUser)
        end = int(end - (pct*fold))
        pctValue=int(end*(pct))
        del readByUser[end-pctValue:end+1]
        for j in range(len(readByUser)):
            element=int(readByUser[j])
            if element < 16979:
                M[i,element]=1
    return M

In [64]:
DF2.tail()

,doc.id,raw.title,raw.abstract
16973,16974,Concentric circles in WMAP data may provide ev...,Conformal cyclic cosmology (CCC) posits the ex...
16974,16975,Analysing biological pathways in genome-wide a...,Genome-wide association (GWA) studies have ty...
16975,16976,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...


In [65]:
DF2.head()

,doc.id,raw.title,raw.abstract
0,1,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,Navigation in a small world,The small-world phenomenon â the principle t...


In [66]:
N=100

In [67]:
preds_df

NameError: name 'preds_df' is not defined

In [76]:
RecList={}
TestList={}

end = len(DF2)

fold=1
    
while fold < 6:
    RecList[fold]=[]
    TestList[fold]=[]
    
    UR_train = getTrain(UR, fold)#np.delete(M, np.s_[end-twpct:end+1], axis=1) # remove columns 1 and 2
    
    U, sigma, Vt = svds(UR_train, k = 500)

    sigma = np.diag(sigma)

    print(U.shape,Vt.shape)

    all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))

    preds_df = pd.DataFrame(all_user_predicted_ratings)
    preds_df.drop(0,inplace=True,axis=1)
    for user_row_number in range(200):#range(len(UR)):
        
        sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False).index.tolist()
        UR_test = getTest(UR.iloc[user_row_number],fold)
        TestList[fold].append(UR_test)
        RecList[fold].append(sorted_user_predictions[0:N])
    fold=fold+1

(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)
(3999, 500) (500, 16979)


In [69]:
'''
recommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].
merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'doc.id',
               right_on = 'index'))

recommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)
recommendations.drop('index',axis=1,inplace=True)
recommendations.sort_values('Predictions', ascending = False,inplace=True)
recommendations.dropna(axis=0,inplace=True)
UR_test = getTest(UR.iloc[user_row_number],fold)
TestList[fold].append(UR_test)
recommendations.reset_index(inplace=True)
'''

"\nrecommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].\nmerge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',\n               left_on = 'doc.id',\n               right_on = 'index'))\n\nrecommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)\nrecommendations.drop('index',axis=1,inplace=True)\nrecommendations.sort_values('Predictions', ascending = False,inplace=True)\nrecommendations.dropna(axis=0,inplace=True)\nUR_test = getTest(UR.iloc[user_row_number],fold)\nTestList[fold].append(UR_test)\nrecommendations.reset_index(inplace=True)\n"

In [ ]:
UR.head()

In [ ]:
def intersect (a,b):
    return list(set(a) & set(b))

In [ ]:
RecList[1][0]

In [ ]:
intersect(RecList[1][0]['doc.id'].values.tolist(),TestList[1][0])

In [ ]:
#display(RecList[1][0]['doc.id'].values.tolist(),UR.iloc[0].values[0],sorted(TestList[1][0],reverse=True))

In [ ]:
BARRIER();

In [ ]:
UR['liked_articles'][215]

In [77]:
#Valor do Prediction nao importa, so se a recomendacao esta no teste ou nao

def DCGp(N, Rel):
    tot = 0
    for i in range(N):
        tot += (2**Rel[i])/mt.log2((i+1)+1)
    return tot

In [78]:
def IDCGp(N, Rel):
    tot = 0
    for i in range(len(Rel)):
        tot += (2**Rel[i])/mt.log2((i+1)+1)
    return tot

In [79]:
def nDCGp(N, Rel):
    
    return (DCGp(N,Rel)/IDCGp(N, Rel))

In [80]:
nDCGList={}
def nDCG11 (RecList, TestList):
    for i in range(len(RecList)):
        i=i+1
        nDCGList[i]=[]
        print('Fold ',i,'\n\n')
        for j in range(len(RecList[i])):
            Rel=[]
            print('Usuario: ',j+1)
            print('Rec:',RecList[i][j]['doc.id'].tolist())
            print('Test:', TestList[i][j])
            
            for x in RecList[i][j]['doc.id'].tolist():
                if x in TestList[i][j]:
                    Rel.append(1)
                else:
                    Rel.append(0)
            
            print('ReL:' ,Rel)
            
            for k in range(N):
                nDCGList[i].append(nDCGp(N, Rel))
            mean=np.mean(nDCGList[i])
            print('nDCG: ',mean,'\n\n')
            del nDCGList[i][:]
            nDCGList[i].append(mean)
            #print(nDCGList[i])
        print('*****************************************')
        mean=np.mean(nDCGList[i])
        print('Fold: ',i,'nDCG mean: ',mean,'\n\n')
        del nDCGList[i][:]
        nDCGList[i].append(mean)

In [81]:
nDCGList={}
def nDCG (RecList, TestList):
    for i in range(len(RecList)):
        i=i+1
        nDCGList[i]=[]
        print('Fold ',i,'\n\n')
        for j in range(len(RecList[i])):
            Rel=[]
            print('Usuario: ',j+1)
            print('Rec:',RecList[i][j])
            print('Test:', TestList[i][j])
            
            for x in RecList[i][j]:
                if x in TestList[i][j]:
                    Rel.append(1)
                else:
                    Rel.append(0)
            
            print('ReL:' ,Rel)
            
            for k in range(N):
                nDCGList[i].append(nDCGp(N, Rel))
            mean=np.mean(nDCGList[i])
            print('nDCG: ',mean,'\n\n')
            del nDCGList[i][:]
            nDCGList[i].append(mean)
            #print(nDCGList[i])
        print('*****************************************')
        mean=np.mean(nDCGList[i])
        print('Fold: ',i,'nDCG mean: ',mean,'\n\n')
        del nDCGList[i][:]
        nDCGList[i].append(mean)

In [82]:
nDCG(RecList, TestList)

Fold  1 


Usuario:  1
Rec: [3404, 70, 6968, 1631, 15165, 5991, 15894, 4871, 12716, 12684, 14676, 10008, 10288, 10204, 8903, 13363, 3297, 13924, 12804, 5614, 495, 12803, 2317, 10508, 10272, 12802, 11650, 3770, 12831, 13187, 11226, 3171, 11009, 13923, 3950, 2846, 45, 3595, 7867, 6103, 2931, 15191, 15282, 15300, 11919, 11320, 16163, 16424, 14662, 2526, 14851, 5114, 5324, 12916, 15026, 6655, 4626, 7801, 5325, 501, 8748, 16089, 12620, 3172, 256, 15913, 14252, 9426, 15330, 15140, 2648, 2676, 9402, 4944, 4061, 7287, 15492, 7918, 7066, 3069, 4139, 6356, 12790, 5936, 6432, 15336, 16588, 4645, 2946, 2321, 6874, 20, 10677, 12064, 4889, 8309, 10287, 11309, 5861, 1532]
Test: [10588, 13172, 4662, 11105, 15336, 12805, 7106, 9907, 3591, 3332, 15833, 11853, 4889, 6874]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

nDCG:  1.0 


Usuario:  10
Rec: [26, 746, 2000, 7786, 14002, 5854, 2095, 830, 3090, 1903, 2439, 3091, 838, 9590, 14329, 8703, 2681, 8346, 6127, 8711, 10507, 840, 13867, 3133, 740, 14162, 8183, 917, 2057, 2460, 2985, 843, 11060, 8223, 8722, 4202, 2691, 3352, 4430, 8451, 834, 6620, 8828, 816, 5002, 8286, 11640, 2470, 2443, 14243, 639, 9835, 1380, 8557, 86, 4899, 11362, 6628, 4150, 833, 13270, 10870, 3185, 10944, 3593, 6798, 1814, 6625, 2468, 857, 1430, 12563, 11252, 1747, 11352, 341, 2473, 6608, 2476, 2474, 11850, 2501, 1861, 6199, 3887, 6751, 8808, 1161, 15129, 11353, 849, 823, 15433, 15835, 3207, 10294, 760, 30, 10117, 8463]
Test: [8469, 7518, 10384, 6620, 7786]
ReL: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 


Usuario: 

Usuario:  27
Rec: [20, 6416, 6848, 13055, 6572, 4018, 5977, 7836, 5919, 11642, 4406, 12825, 11743, 10819, 10240, 13472, 4620, 12974, 5346, 9970, 1712, 12771, 15029, 13337, 6847, 4017, 12778, 10890, 12777, 12204, 3481, 5701, 4019, 428, 8385, 446, 3970, 6454, 8586, 14026, 12979, 441, 8848, 16320, 5918, 10148, 15246, 15904, 15355, 15646, 5745, 12984, 5312, 10019, 8600, 13412, 14367, 2821, 1700, 10869, 13800, 2415, 51, 9387, 3846, 6419, 768, 5223, 14027, 12770, 7025, 9842, 3943, 12475, 7075, 4587, 3342, 867, 7780, 6414, 3071, 10520, 13629, 14333, 2739, 4142, 6062, 10763, 10279, 3617, 445, 10177, 15953, 8851, 2719, 13798, 11566, 10448, 12392, 5818]
Test: [8901, 5977, 5745, 11743]
ReL: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0

nDCG:  1.0 


Usuario:  45
Rec: [16, 17, 3981, 1200, 2216, 477, 34, 1796, 1226, 3428, 1706, 4854, 9290, 11205, 11164, 216, 3912, 4020, 2038, 3419, 511, 11567, 3422, 4652, 2461, 226, 5273, 10416, 416, 12453, 3556, 6495, 2206, 1400, 3429, 3783, 3484, 3420, 2469, 6091, 3215, 8791, 11203, 9496, 3433, 142, 144, 2794, 2622, 1357, 417, 656, 8955, 1595, 1957, 2350, 189, 3914, 5211, 7135, 8297, 7154, 1004, 5267, 4843, 7605, 11198, 9962, 2496, 3138, 1546, 4281, 3413, 6038, 4651, 3424, 6370, 247, 4650, 243, 1002, 1142, 4742, 6544, 10659, 3808, 6403, 12563, 6832, 4282, 1940, 1192, 11189, 32, 5526, 8318, 2360, 8511, 1035, 2769]
Test: [226, 6766, 2038, 14, 380, 8050, 9400]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 


Usuario:  46

nDCG:  1.0 


Usuario:  63
Rec: [29, 6653, 13247, 13308, 13912, 11610, 16311, 8464, 16760, 14035, 8204, 12133, 10418, 16408, 6051, 9506, 9353, 6952, 5207, 11843, 13253, 16694, 11563, 8517, 16382, 12970, 10827, 16632, 13793, 12040, 12726, 10742, 7418, 13146, 4695, 5597, 11666, 5371, 11570, 8460, 11138, 8591, 9610, 16477, 16468, 10863, 11588, 2277, 9206, 2278, 790, 11152, 11915, 13338, 13023, 595, 2837, 5839, 568, 4122, 12971, 4502, 15114, 10817, 6301, 5836, 3815, 11179, 10907, 786, 13854, 13816, 5645, 8189, 8082, 14180, 12760, 2198, 16384, 12408, 12621, 5994, 15473, 6862, 6297, 2987, 97, 11589, 12940, 1409, 7185, 16206, 7790, 11272, 16924, 3614, 10848, 8353, 11569, 12850]
Test: [12981, 12702, 13570, 10848, 11915, 13290]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

nDCG:  1.0 


Usuario:  84
Rec: [10, 1203, 2351, 837, 1665, 853, 1974, 6098, 1204, 752, 1206, 1179, 7147, 842, 9030, 1700, 33, 846, 830, 1161, 9888, 239, 8097, 436, 821, 934, 4171, 829, 40, 30, 827, 958, 1152, 4045, 6920, 3336, 1944, 3414, 8595, 5261, 46, 5737, 11848, 8707, 4080, 825, 1182, 2781, 835, 1812, 980, 1261, 1208, 418, 1194, 1902, 12074, 2411, 7468, 43, 7127, 3343, 2472, 1250, 2609, 1220, 2214, 3290, 834, 1650, 4808, 11199, 13421, 5429, 1221, 1016, 6182, 11299, 14497, 4134, 72, 4820, 9998, 849, 5148, 6944, 12128, 779, 2164, 37, 8493, 941, 1199, 7242, 856, 2233, 9738, 16045, 7033, 3605]
Test: [2066, 12715, 8733]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 


Usuario:  85
Rec: [39, 4080, 2715, 5120, 3699, 2479

nDCG:  1.0 


Usuario:  113
Rec: [5120, 369, 443, 44, 2445, 76, 45, 1117, 4040, 615, 906, 447, 2124, 1118, 192, 1469, 8628, 6390, 6826, 5516, 1225, 1434, 8754, 451, 1778, 217, 4148, 8779, 3233, 1285, 5684, 2956, 3118, 4847, 4532, 265, 2631, 3981, 2547, 3510, 6825, 139, 1263, 316, 3408, 3593, 718, 1924, 2988, 4445, 3527, 10737, 557, 2082, 7084, 7808, 4953, 527, 163, 1326, 6189, 1327, 6526, 120, 2389, 4367, 1401, 1372, 881, 1371, 280, 11548, 2546, 3080, 266, 3234, 7898, 2637, 1286, 1278, 6761, 1348, 117, 2057, 11322, 5732, 1428, 8014, 2984, 4792, 2324, 7886, 6248, 348, 13258, 11169, 13014, 2098, 3026, 4726]
Test: [3026, 4073, 2391, 4726, 5034, 11660, 10200, 2712, 5221, 198, 7886, 167, 162, 7995, 423, 7416, 3339, 398, 6522, 4444, 1419, 10678, 2098, 1997]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

nDCG:  1.0 


Usuario:  141
Rec: [15874, 15981, 16700, 15838, 16706, 15944, 16959, 15531, 15633, 15614, 15263, 15699, 16288, 15609, 16358, 16900, 16503, 15670, 16868, 14935, 16304, 9241, 16907, 16192, 16099, 16383, 16141, 16434, 16529, 16723, 14647, 15804, 15655, 16179, 16618, 16749, 16819, 15484, 14285, 9609, 16084, 12085, 15930, 116, 16431, 16845, 16831, 15737, 15736, 15858, 15993, 16272, 15882, 15888, 15816, 16754, 16449, 15852, 16303, 16096, 16186, 16235, 12290, 14646, 16671, 16901, 16316, 16339, 16107, 12422, 16909, 16276, 16530, 16921, 16185, 16739, 15837, 16450, 16552, 16047, 16243, 16848, 16261, 16330, 16933, 16862, 16350, 16323, 16277, 15420, 16581, 16162, 16960, 16719, 16140, 49, 16850, 15768, 14230, 13785]
Test: [15665, 16016, 14399, 14988, 16131, 9228, 16682, 15856, 16077, 16365, 16409, 16829, 15710, 16520, 15373, 15197, 16880, 16519, 16292, 14122, 16064, 16654, 15890, 16658]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

nDCG:  1.0 


Usuario:  169
Rec: [24, 6653, 448, 667, 6869, 1451, 6867, 1168, 6694, 7963, 7821, 9199, 6652, 9198, 7964, 1766, 6870, 7790, 9202, 9859, 5926, 9610, 7529, 6511, 13457, 14699, 11965, 10164, 2851, 7788, 16206, 4629, 4851, 16306, 5153, 16384, 4753, 563, 5521, 694, 1671, 1555, 7311, 11601, 8176, 7912, 10345, 11283, 3549, 4278, 554, 10239, 5203, 4138, 9900, 13984, 138, 7665, 51, 9413, 7418, 7530, 8777, 13338, 3291, 588, 2830, 4211, 4557, 3910, 5722, 6246, 16233, 6432, 3577, 11036, 10493, 3380, 10925, 11857, 5606, 13846, 4923, 2642, 4089, 11221, 715, 9678, 624, 9442, 6528, 12015, 10202, 15138, 11742, 6845, 716, 12404, 1747, 1616]
Test: [6868, 715, 6866, 1766, 6870]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
nDCG:  1.0 



nDCG:  1.0 


Usuario:  196
Rec: [14607, 16318, 12244, 6359, 16959, 14938, 16165, 12790, 54, 15526, 16868, 16280, 16438, 16900, 16319, 15773, 16870, 16501, 16349, 15485, 8748, 16664, 16575, 16034, 16424, 16748, 15815, 16564, 15863, 16918, 16863, 6570, 16151, 15033, 16264, 16161, 15113, 14758, 16914, 15514, 15598, 14851, 15738, 16595, 16223, 14726, 16371, 16043, 6874, 11236, 16335, 16898, 15944, 15487, 16475, 16152, 13979, 10008, 16622, 16358, 6235, 14447, 12831, 12523, 16423, 16509, 8128, 16416, 13130, 8903, 15207, 13277, 14676, 6294, 12537, 15760, 16524, 16687, 8531, 11650, 12468, 16838, 26, 10423, 15358, 16721, 14285, 15165, 9234, 33, 42, 10891, 13853, 16896, 12410, 16788, 12290, 15267, 12754, 5804]
Test: [15661, 16386, 16789, 16560, 15481, 14988, 16296, 16469, 16495, 16896, 16545]
ReL: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [83]:
nDCGList

{1: [1.0], 2: [1.0], 3: [1.0], 4: [1.0], 5: [1.0]}

In [ ]:
sorted(RecList[1][0]['doc.id'].tolist())

In [ ]:
sorted(TestList[1][0])

In [ ]:
nDCGList

In [ ]:
nDCGList[1]

### Hybrid

In [ ]:
def getTFIDFUser(rowNumber):
    AllString=''
    for element in UR.iloc[rowNumber]['liked_articles']:
        element=int(element)
        rawAbstract=DF2[DF2['doc.id']==element]['raw.abstract'].values[0]
        AllString=AllString+rawAbstract
    return AllString

In [ ]:
DF2.info()